In [ ]:
!pip -qq install torch
!pip -qq install torchtext
!pip -qq install spacy
!python -m spacy download en_core_web_sm
!pip -qq install sacremoses==0.0.5
!pip -qq install subword_nmt==0.3.5
!wget http://www.manythings.org/anki/rus-eng.zip 
!unzip rus-eng.zip

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Machine Translation

## Подготовка данных

Начнем с чтения данных. Возьмем их у anki, поэтому они немного специфичны:

In [ ]:
!shuf -n 10 rus.txt

Токенизируем их:

In [ ]:
from torchtext.legacy.data import Field, Example, Dataset, BucketIterator

BOS_TOKEN = '<s>'
EOS_TOKEN = '</s>'

source_field = Field(tokenize='spacy', init_token=None, eos_token=EOS_TOKEN)
target_field = Field(tokenize='moses', init_token=BOS_TOKEN, eos_token=EOS_TOKEN)
fields = [('source', source_field), ('target', target_field)]

In [ ]:
source_field.preprocess("It's surprising that you haven't heard anything about her wedding.")

In [ ]:
target_field.preprocess('Удивительно, что ты ничего не слышал о её свадьбе.')

In [ ]:
from tqdm import tqdm

MAX_TOKENS_COUNT = 16
SUBSET_SIZE = .3

examples = []
with open('rus.txt', encoding='utf8') as f:
    for line in tqdm(f, total=440219):
        source_text, target_text, _ = line.split('\t')
        source_text = source_field.preprocess(source_text)
        target_text = target_field.preprocess(target_text)
        if len(source_text) <= MAX_TOKENS_COUNT and len(target_text) <= MAX_TOKENS_COUNT:
            if np.random.rand() < SUBSET_SIZE:
                examples.append(Example.fromlist([source_text, target_text], fields))

Построим датасеты:

In [ ]:
dataset = Dataset(examples, fields)

train_dataset, test_dataset = dataset.split(split_ratio=0.85)

print('Train size =', len(train_dataset))
print('Test size =', len(test_dataset))

source_field.build_vocab(train_dataset, min_freq=3)
print('Source vocab size =', len(source_field.vocab))

target_field.build_vocab(train_dataset, min_freq=3)
print('Target vocab size =', len(target_field.vocab))

train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), batch_sizes=(32, 256), shuffle=True, device=DEVICE, sort=False
)

In [ ]:
source_field.process([source_field.preprocess("It's surprising that you haven't heard anything about her wedding.")])

In [ ]:
source_field.vocab.itos

In [ ]:
target_field.vocab.itos

## Seq2seq модель

Пора писать простой seq2seq. Разобьем модель на несколько модулей - Encoder, Decoder и их объединение. 

Encoder должен быть подобен символьной сеточке в POS tagging'е: эмбеддить токены и запускать rnn'ку (в данном случае будем пользоваться GRU) и отдавать последнее скрытое состояние.

Decoder почти такой же, только еще и предсказывает токены на каждом своем шаге.

**Задание** Реализовать модели.

In [ ]:
batch = next(iter(train_iter))

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()
        
        self._bidir = False
        self._num_layers = num_layers
        self._hidden_dim = rnn_hidden_dim // 2 if self._bidir else rnn_hidden_dim
        
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=self._hidden_dim, 
                           num_layers=num_layers, bidirectional=self._bidir, dropout=0.2)

    def forward(self, inputs, hidden=None):
        # ???

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=rnn_hidden_dim, num_layers=num_layers)
        self._out = nn.Linear(rnn_hidden_dim, vocab_size)

    def forward(self, inputs, hidden=None):
        # ???

Модель перевода будет просто сперва вызывать Encoder, а потом передавать его скрытое состояние декодеру в качестве начального.

In [ ]:
class TranslationModel(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, emb_dim=128, 
                 rnn_hidden_dim=256, encoder_layers=2, decoder_layers=1):
        
        super().__init__()
        
        self.encoder = Encoder(source_vocab_size, emb_dim, rnn_hidden_dim, num_layers=encoder_layers)
        self.decoder = Decoder(target_vocab_size, emb_dim, rnn_hidden_dim, num_layers=decoder_layers)
        
    def forward(self, source_inputs, target_inputs):
        encoder_hidden = self.encoder(source_inputs)
        
        return self.decoder(target_inputs, encoder_hidden)

In [ ]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), target_vocab_size=len(target_field.vocab),
                        encoder_layers=2).to(DEVICE)

outs = model(batch.source, batch.target)
outs[0].shape, outs[1].shape

Реализуем простой перевод - жадный. На каждом шаге будем выдавать самый вероятный из предсказываемых токенов:

![](https://github.com/tensorflow/nmt/raw/master/nmt/g3doc/img/greedy_dec.jpg)  
*From [tensorflow/nmt](https://github.com/tensorflow/nmt)*

**Задание** Реализовать функцию.

In [ ]:
def greedy_decode(model, source_text, source_field, target_field):
    bos_index = target_field.vocab.stoi[BOS_TOKEN]
    eos_index = target_field.vocab.stoi[EOS_TOKEN]
    
    model.eval()
    with torch.no_grad():
        result = [] # list of predicted tokens indices
        # ???
            
        return ' '.join(target_field.vocab.itos[ind.squeeze().item()] for ind in result)

In [ ]:
greedy_decode(model, "Do you believe?", source_field, target_field)

Нужно как-то оценивать модель.

Обычно для этого используется [BLEU скор](https://en.wikipedia.org/wiki/BLEU) - что-то вроде точности угадывания n-gram из правильного (референсного) перевода.

**Задание** Реализовать функцию оценки: для батчей из `iterator` предсказать их переводы, обрезать по `</s>` и сложить правильные варианты и предсказанные в `refs` и `hyps` соответственно.

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def evaluate_model(model, iterator):
    model.eval()
    refs, hyps = [], []
    bos_index = iterator.dataset.fields['target'].vocab.stoi[BOS_TOKEN]
    eos_index = iterator.dataset.fields['target'].vocab.stoi[EOS_TOKEN]
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            encoder_hidden = model.encoder(batch.source)

            hidden = encoder_hidden
            result = [LongTensor([bos_index]).expand(1, batch.target.shape[1])]

            for _ in range(30):
                step, hidden = model.decoder(result[-1], hidden)
                step = step.argmax(-1)
                result.append(step)

            targets = batch.target.data.cpu().numpy().T
            _, eos_indices = np.where(targets == eos_index)

            targets = [target[:eos_ind] for eos_ind, target in zip(eos_indices, targets)]

            refs.extend(targets)

            result = torch.cat(result)
            result = result.data.cpu().numpy().T
            _, eos_indices = np.where(result == eos_index)
            result = [res[:eos_ind] for eos_ind, res in zip(eos_indices, result)]
            hyps.extend(result)
            
    return corpus_bleu([[ref] for ref in refs], hyps) * 100

In [ ]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.source, batch.target)
                
                target = torch.cat((batch.target[1:], batch.target.new_ones((1, batch.target.shape[1]))))
                loss = criterion(logits.view(-1, logits.shape[-1]), target.view(-1))

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, None, name_prefix + '  Val:')
            print('\nVal BLEU = {:.2f}'.format(evaluate_model(model, val_iter)))

In [ ]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), target_vocab_size=len(target_field.vocab), encoder_layers=2).to(DEVICE)

pad_idx = target_field.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)

optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=30, val_iter=test_iter)

In [ ]:
evaluate_model(model, test_iter)

In [ ]:
greedy_decode(model, "Hello, world", source_field, target_field)

## Реализация attention'а

В общем случае, attention работает так: пусть у нас есть набор скрытых состояний $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - представлений слов из исходного языка, полученных с помощью энкодера. И есть некоторое текущее скрытое состояние $\mathbf{h}_i$ - скажем, представление, используемое для предсказания слова на нужном нам языке.

Тогда с помощью аттеншена мы можем получить взвешенное представление контекста $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - вектор $\mathbf{c}_i$:
$$
\begin{align}\begin{split}
\mathbf{c}_i &= \sum\limits_j a_{ij}\mathbf{s}_j\\
\mathbf{a}_{ij} &= \text{softmax}(f_{att}(\mathbf{h}_i, \mathbf{s}_j))
\end{split}\end{align}
$$

$f_{att}$ - функция, которая говорит, насколько хорошо $\mathbf{h}_i$ и $\mathbf{s}_j$ подходят друг другу.

Самые популярные её варианты:
- Additive attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{v}_a{}^\top \text{tanh}(\mathbf{W}_a\mathbf{h}_i + \mathbf{W}_b\mathbf{s}_j)$$
- Dot attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{s}_j$$
- Multiplicative attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{W}_a \mathbf{s}_j$$

**Задание** Реализуйте Additive attention.

In [ ]:
class AdditiveAttention(nn.Module):
    def __init__(self, query_size, key_size, hidden_dim):
        super().__init__()

        # query - decoder state, (1, batch, rnn_hidden_dim)
        # value - all encoder states, (seq_len, batch, rnn_hidden_dim)
        # key_proj - self.key_layer(value), (seq_len, batch, hidden_dim)
        # hidden_dim - size of attention layer
        
        self._query_layer = nn.Linear(query_size, hidden_dim)
        self._key_layer = nn.Linear(key_size, hidden_dim)
        self._energy_layer = nn.Linear(hidden_dim, 1)
        
    def forward(self, query, key_proj, value, mask=None):
        # ???

Обновим декодер

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()
        
        self._bidir = False
        self._num_layers = num_layers
        self._hidden_dim = rnn_hidden_dim // 2 if self._bidir else rnn_hidden_dim
        
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=self._hidden_dim, 
                           num_layers=num_layers, bidirectional=self._bidir, dropout=0.2)

    def forward(self, inputs, hidden=None):
        embs = self._emb(inputs)
        # seq_len, batch_size, 1
        outputs, h = self._rnn(embs, hidden) # у GRU нет h_c, а output нам не нужен
        return outputs, h[-1].unsqueeze(0)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=64, rnn_hidden_dim=256, attn_dim=128, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim + rnn_hidden_dim, 
                           hidden_size=rnn_hidden_dim, num_layers=num_layers)
        self._out = nn.Linear(rnn_hidden_dim, vocab_size) 
        self._att = AdditiveAttention(rnn_hidden_dim, rnn_hidden_dim, attn_dim)
        self._drop = nn.Dropout(p=0.3)

    def forward(self, inputs, encoder_output, encoder_mask, hidden=None):
        embs = self._emb(inputs)
        outputs = []
        attentions = []
        key_proj = self._att._key_layer(encoder_output) # можно попробовать вынести в attention
        for i in range(embs.shape[0]):
            context, f_att = self._att(query=hidden, key_proj=key_proj,
                                       value=encoder_output, mask=encoder_mask)
            context = context.unsqueeze(0)
            rnn_input = torch.cat((embs[i:i + 1], context), -1)
            output, hidden = self._rnn(rnn_input, hidden)

            outputs.append(output)
            attentions.append(f_att)

        output = self._drop(torch.cat(outputs))
        attentions = torch.cat(attentions)
        return self._out(output), hidden, attentions

In [ ]:
class TranslationModel(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, emb_dim=64, rnn_hidden_dim=128, 
                 attn_dim=128, encoder_num_layers=2):
        
        super().__init__()
        
        self.encoder = Encoder(source_vocab_size, emb_dim, rnn_hidden_dim, encoder_num_layers)
        self.decoder = Decoder(target_vocab_size, emb_dim, rnn_hidden_dim, attn_dim, 1)
        
    def forward(self, source_inputs, target_inputs):
        encoder_mask = source_inputs == 1
        encoder_output, encoder_hidden = self.encoder(source_inputs)
        return self.decoder(target_inputs, encoder_output, encoder_mask, encoder_hidden)

In [ ]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), target_vocab_size=len(target_field.vocab)).to(DEVICE)

model(batch.source, batch.target)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def evaluate_model(model, iterator):
    model.eval()
    refs, hyps = [], []
    bos_index = iterator.dataset.fields['target'].vocab.stoi[BOS_TOKEN]
    eos_index = iterator.dataset.fields['target'].vocab.stoi[EOS_TOKEN]
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            encoder_output, encoder_hidden = model.encoder(batch.source)
            mask = batch.source == 1.
            
            hidden = encoder_hidden
            result = [LongTensor([bos_index]).expand(1, batch.target.shape[1])]
            
            for _ in range(30):
                step, hidden, _ = model.decoder(result[-1], encoder_output, mask, hidden)
                step = step.argmax(-1)
                result.append(step)
            
            targets = batch.target.data.cpu().numpy().T
            eos_indices = (targets == eos_index).argmax(-1)
            eos_indices[eos_indices == 0] = targets.shape[1]

            targets = [target[:eos_ind] for eos_ind, target in zip(eos_indices, targets)]
            refs.extend(targets)
            
            result = torch.cat(result)
            result = result.data.cpu().numpy().T
            eos_indices = (result == eos_index).argmax(-1)
            eos_indices[eos_indices == 0] = result.shape[1]

            result = [res[:eos_ind] for eos_ind, res in zip(eos_indices, result)]
            hyps.extend(result)
            
    return corpus_bleu([[ref] for ref in refs], hyps) * 100


import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _, _ = model(batch.source, batch.target)
                
                target = torch.cat((batch.target[1:], batch.target.new_ones((1, batch.target.shape[1]))))
                loss = criterion(logits.view(-1, logits.shape[-1]), target.view(-1))

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, None, name_prefix + '  Val:')
            print('\nVal BLEU = {:.2f}'.format(evaluate_model(model, val_iter)))

In [ ]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), target_vocab_size=len(target_field.vocab)).to(DEVICE)

pad_idx = target_field.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)

optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=30, val_iter=test_iter)

In [ ]:
def greedy_decode(model, source_text, source_field, target_field):
    bos_index = target_field.vocab.stoi[BOS_TOKEN]
    eos_index = target_field.vocab.stoi[EOS_TOKEN]
    
    model.eval()
    with torch.no_grad():
        result, attentions = [], []
        source = source_field.preprocess(source_text)
        inputs = source_field.process([source]).to(DEVICE)
        
        encoder_output, encoder_hidden = model.encoder(inputs)
        encoder_mask = torch.zeros_like(inputs).byte()
        
        hidden = encoder_hidden
        step = LongTensor([[bos_index]])
        
        for _ in range(50):
            step, hidden, attention = model.decoder(step, encoder_output, encoder_mask, hidden)
            step = step.argmax(-1)
            attentions.append(attention.squeeze(1))
          
            if step.squeeze().item() == eos_index:
                break
            
            result.append(step.item())   
        result = [target_field.vocab.itos[ind] for ind in result]
        return source, result, torch.cat(attentions, -1).data.cpu().numpy()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_heatmap(src, trg, scores):

    fig, ax = plt.subplots()
    heatmap = ax.pcolor(scores, cmap='viridis')

    ax.set_xticklabels(trg, minor=False, rotation=45)
    ax.set_yticklabels(src, minor=False)

    ax.xaxis.tick_top()
    ax.set_xticks(np.arange(scores.shape[1]) + 0.5, minor=False)
    ax.set_yticks(np.arange(scores.shape[0]) + 0.5, minor=False)
    ax.invert_yaxis()

    plt.colorbar(heatmap)
    plt.show()

In [ ]:
source, result, attentions = greedy_decode(model, "I didn't pay.", source_field, target_field)

In [ ]:
plot_heatmap(source + ['</s>'], result + ['</s>'], attentions)